### Import Required Libraries and Set Up Environment Variables

In [43]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
query='love'
filter_query = 'section_name:("Movies") AND type_of_material:("Review") AND headline:("love")'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url=f"{url}api-key={nyt_api_key}&q={query}&fq={filter_query}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&sort={sort}"

In [4]:
#Check
page_url=query_url
articles = requests.get(query_url).json()
articles_list = articles["response"]["docs"]
print(json.dumps(articles_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
    # create query with a page number
    # API results show 10 articles at a time

for page in range(0,20):
    page_url=f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    articles = requests.get(page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        articles = articles["response"]["docs"]
        # loop through the reviews["response"]["docs"] and append each review to the list
        for article in articles:
            reviews_list.append(article)
            content=len(reviews_list)
        # Print the page that was just retrieved
        print(f"the page number retrieved is:{page}") 
        
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"There is an error retrieving articles on page :{page}") 
        break
#print(json.dumps(reviews_list, indent=4))

the page number retrieved is:0
the page number retrieved is:1
the page number retrieved is:2
the page number retrieved is:3
the page number retrieved is:4
the page number retrieved is:5
the page number retrieved is:6
the page number retrieved is:7
the page number retrieved is:8
the page number retrieved is:9
the page number retrieved is:10
the page number retrieved is:11
the page number retrieved is:12
the page number retrieved is:13
the page number retrieved is:14
the page number retrieved is:15
the page number retrieved is:16
the page number retrieved is:17
the page number retrieved is:18
the page number retrieved is:19


In [9]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range (0,5):
    print(json.dumps(reviews_list[i], indent=4))

{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [15]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [42]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df = pd.json_normalize(reviews_list)
' Added a - before Review to make sure it reads like a title '
' Some titles had '' and : witihn so adjusted for it by adding headline 2 to modify the string and then replacing them'
#reviews_df['headline.main2']=reviews_df['headline.main'].str.replace("’ Review:","++")
#reviews_df['headline.main2']=reviews_df['headline.main'].str.replace("Review: ",'')
reviews_df['title']=reviews_df['headline.main'].str.extract(r"\u2018(.*?)\u2019", expand=True)
#reviews_df=reviews_df.drop('headline.main2', axis =1)

reviews_df[['headline.main','title']]

,headline.main,title
0,"‘The Attachment Diaries’ Review: Love, Sick",The Attachment Diaries
1,Review: ‘What’s Love Got to Do With It?’ Proba...,What
2,‘You Can Live Forever’ Review: Do You Love Me ...,You Can Live Forever
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,A Tourist
4,‘Other People’s Children’ Review: True Romance,Other People
...,...,...
195,Review: A Combustible Pair Find Love in ‘The O...,The Other Half
196,"Review: Love as the World Wars, in ‘The Ottoma...",The Ottoman Lieutenant
197,Review: It’s All Mirth and Taxes in ‘Love & Ta...,Love & Taxes
198,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...","Everybody Loves Somebody,"


In [28]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}:{item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords2']= reviews_df['keywords'].apply(extract_keywords)
reviews_df.head(10)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,keywords2
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,subject:Movies;creative_works:The Attachment D...
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"subject:Movies;persons:Kapur, Shekhar;persons:..."
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,subject:Movies;creative_works:You Can Live For...
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist,subject:Movies;creative_works:A Tourist's Guid...
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People,"subject:Movies;persons:Zlotowski, Rebecca;crea..."
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"subject:Movies;persons:Bracey, Luke (1989- );p..."
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story,subject:Documentary Films and Programs;creativ...
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One,subject:Movies;creative_works:A Thousand and O...
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': '

In [29]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
reviews_df['title_main']=reviews_df['title'].str.replace('- A Review','')
title_list = reviews_df['title_main'].tolist()
title_list

['The Attachment Diaries',
 'What',
 'You Can Live Forever',
 'A Tourist',
 'Other People',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie',
 'Love Is Love 

### Access The Movie Database API

In [32]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [48]:
#check
#for title in title_list:
#    query_url=f"{url}{title}{tmdb_key_string}"
 #   articles = requests.get(query_url).json()
  #  print(json.dumps(articles, indent=4))

# id check 
id =224609
movie_query_url = f"https://api.themoviedb.org/3/movie/{id}?{tmdb_key_string}"
art = requests.get(movie_query_url).json()
print(json.dumps(art, indent=4))

{
    "adult": false,
    "backdrop_path": "/3BIi5hEsGTJ17tVgdD6c9JWYnWJ.jpg",
    "belongs_to_collection": null,
    "budget": 0,
    "genres": [
        {
            "id": 10749,
            "name": "Romance"
        }
    ],
    "homepage": "",
    "id": 224609,
    "imdb_id": "tt1966496",
    "origin_country": [
        "IN"
    ],
    "original_language": "ta",
    "original_title": "\u0b95\u0bc7\u0b9f\u0bbf",
    "overview": "A happy-go-lucky student at a university gets embroiled in a love triangle with two very different young women.",
    "popularity": 5.464,
    "poster_path": "/3IP6uwxDSG3GqDnT1ffRLFtsjDx.jpg",
    "production_companies": [
        {
            "id": 4311,
            "logo_path": null,
            "name": "Sri Surya Movies",
            "origin_country": "IN"
        }
    ],
    "production_countries": [
        {
            "iso_3166_1": "IN",
            "name": "India"
        }
    ],
    "release_date": "2006-09-24",
    "revenue": 0,
    "runtime"

In [ ]:
# Create an empty list to store the results
movie_list=[]
# Create a request counter to sleep the requests after a multiple
# of 50 requests
counter=0
# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if counter>=50:
        time.sleep(10)
     # Add 1 to the request counter
    counter=counter+1
    try:
    # Perform a "GET" request for The Movie Database
        query_url=f"{url}{title}{tmdb_key_string}"
        articles = requests.get(query_url).json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id
        movie_id=articles['results'][0]['id']

        # Make a request for a the full movie details
        movie_query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}"

        # Execute "GET" request with url
        movie_details = requests.get(movie_query_url).json()
        
        # Extract the genre names into a list
        genres=[genre['name'] for genre in movie_details['genres']]

        # Extract the spoken_languages' English name into a list
        lang=[lang['english_name'] for lang in movie_details['spoken_languages']]

        # Extract the production_countries' name into a list

        prod_country=[country['name'] for country in movie_detail['production_countries']]
        # Add the relevant data to a dictionary and
        movie_key_details ={
                            'Title': movie_details['title'],
                            'Genre': genres,
                            'Language': lang,
                            'Country': prod_country
                            }
        # append it to the tmdb_movies_list list

        movie_list.append(movie_key_details)
        # Print out the title that was found
        print(f" Movie {title} details in tmdb")
    except:
        print(f" Movie {title} details not found tmdb")

 Movie The Attachment Diaries details not found tmdb
 Movie What details not found tmdb
 Movie You Can Live Forever details not found tmdb
 Movie A Tourist details not found tmdb
 Movie Other People details not found tmdb
 Movie One True Loves details not found tmdb
 Movie The Lost Weekend: A Love Story details not found tmdb
 Movie A Thousand and One details not found tmdb
 Movie Your Place or Mine details not found tmdb
 Movie Love in the Time of Fentanyl details not found tmdb
 Movie Pamela, a Love Story details not found tmdb
 Movie In From the Side details not found tmdb
 Movie After Love details not found tmdb
 Movie Alcarràs details not found tmdb
 Movie Nelly & Nadine details not found tmdb
 Movie Lady Chatterley details not found tmdb
 Movie The Sound of Christmas details not found tmdb
 Movie The Inspection details not found tmdb
 Movie Bones and All details not found tmdb
 Movie My Policeman details not found tmdb
 Movie About Fate details not found tmdb
 Movie Waiting for B

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [ ]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
